In [1]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
# !unzip cased_L-12_H-768_A-12.zip

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

In [4]:
BERT_VOCAB = 'cased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'cased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'cased_L-12_H-768_A-12/bert_config.json'

tokenization.validate_case_matches_checkpoint(True, '')
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=False)
MAX_SEQ_LENGTH = 120

In [5]:
import json

with open('text.json') as fopen:
    data = json.load(fopen)
    
data.keys()

dict_keys(['train_X', 'train_Y', 'test_X', 'test_Y'])

In [6]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
              tokens_a.pop()
        else:
              tokens_b.pop()

def get_data(left, right):
    input_ids, input_masks, segment_ids = [], [], []
    for i in tqdm(range(len(left))):
        tokens_a = tokenizer.tokenize(left[i])
        tokens_b = tokenizer.tokenize(right[i])
        _truncate_seq_pair(tokens_a, tokens_b, MAX_SEQ_LENGTH - 3)
        tokens = []
        segment_id = []
        tokens.append("[CLS]")
        segment_id.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_id.append(0)
        tokens.append("[SEP]")
        segment_id.append(0)
        for token in tokens_b:
            tokens.append(token)
            segment_id.append(1)
        tokens.append("[SEP]")
        segment_id.append(1)
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)

        while len(input_id) < MAX_SEQ_LENGTH:
            input_id.append(0)
            input_mask.append(0)
            segment_id.append(0)

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    return input_ids, input_masks, segment_ids

In [7]:
left, right = [], []
for i in range(len(data['train_X'])):
    l, r = data['train_X'][i].split(' <> ')
    left.append(l)
    right.append(r)
    
train_ids, train_masks, segment_train = get_data(left, right)

100%|██████████| 261802/261802 [02:41<00:00, 1625.51it/s]


In [8]:
left, right = [], []
for i in range(len(data['test_X'])):
    l, r = data['test_X'][i].split(' <> ')
    left.append(l)
    right.append(r)
    
test_ids, test_masks, segment_test = get_data(left, right)

100%|██████████| 13395/13395 [00:10<00:00, 1282.59it/s]


In [9]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [10]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(left) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [11]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=True,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from cased_L-12_H-768_A-12/bert_model.ckpt


In [13]:
labels = ['contradiction', 'entailment']

train_Y = data['train_Y']
test_Y = data['test_Y']

train_Y = [labels.index(i) for i in train_Y]
test_Y = [labels.index(i) for i in test_Y]

In [14]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_ids))
        batch_x = train_ids[i: index]
        batch_masks = train_masks[i: index]
        batch_segment = segment_train[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    pbar = tqdm(range(0, len(test_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_ids))
        batch_x = test_ids[i: index]
        batch_masks = test_masks[i: index]
        batch_segment = segment_test[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/4364 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.911533
time taken: 2273.8655416965485
epoch: 0, training loss: 0.258576, training acc: 0.892458, valid loss: 0.220598, valid acc: 0.911533



train minibatch loop:   7%|▋         | 324/4364 [02:45<34:26,  1.96it/s, accuracy=0.95, cost=0.133]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

test minibatch loop: 100%|██████████| 224/224 [00:40<00:00,  5.55it/s, accuracy=1, cost=0.0818]    

time taken: 2270.5256547927856
epoch: 1, training loss: 0.197927, training acc: 0.923370, valid loss: 0.220682, valid acc: 0.908110

break epoch:2



In [17]:
tqdm._instances.clear()

In [18]:
test_acc, test_loss = [], []

pbar = tqdm(range(0, len(test_ids), batch_size), desc = 'test minibatch loop')
for i in pbar:
    index = min(i + batch_size, len(test_ids))
    batch_x = test_ids[i: index]
    batch_masks = test_masks[i: index]
    batch_segment = segment_test[i: index]
    batch_y = test_Y[i: index]
    acc, cost = sess.run(
        [model.accuracy, model.cost],
        feed_dict = {
            model.Y: batch_y,
            model.X: batch_x,
            model.segment_ids: batch_segment,
            model.input_masks: batch_masks
        },
    )
    test_loss.append(cost)
    test_acc.append(acc)
    pbar.set_postfix(cost = cost, accuracy = acc)

test minibatch loop: 100%|██████████| 224/224 [00:40<00:00,  5.57it/s, accuracy=1, cost=0.0616]   


In [19]:
test_loss = np.mean(test_loss)
test_acc = np.mean(test_acc)
test_loss, test_acc

(0.22157677, 0.91123515)